In [ ]:
import sys
import os
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils.environment_validation import validate_environment, validate_model_access
validate_environment()

In [ ]:
required_models = [
    "amazon.titan-embed-text-v1",
    "us.anthropic.claude-3-5-haiku-20241022-v1:0",
    "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    "us.amazon.nova-pro-v1:0",
]
validate_model_access(required_models)

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel, GradioUI, tool
from typing import List
import pandas as pd
import yfinance as yf
import pandas_datareader as pdr
import statsmodels.api as sm
import numpy as np


In [ ]:
MODEL_ID = "bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0"
model = LiteLLMModel(model_id=MODEL_ID, temperature=0)

In [ ]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model)
agent.run("How many seconds would it take for a leopard at full speed to run through Pont des Arts?")

In [ ]:
@tool
def get_ticker_data(
    tickers: List[str],
    start_date: str,
    end_date: str,
    metric: str = "all",
    sampling: str = "monthly",
) -> dict:
    """Downloads historical stock data from Yahoo Finance and returns it as a dictionary.

    Examples:
        >>> get_ticker_data(["AAPL"], "2023-01-01", "2023-12-31", "Close", "weekly")
        {"AAPL": [{"Date": "2023-01-06", "Close": 129.619995}, {"Date": "2023-01-13", "Close": 134.759995}, ...]}

        >>> get_ticker_data(["AAPL", "MSFT"], "2023-01-01", "2023-12-31", "all", "monthly")
        {"AAPL": [{"Date": "2023-01-31", "Open": 144.479996, "High": 147.229996, "Low": 141.320007, "Close": 144.289993, "Adj Close": 143.839996, "Volume": 77663600}, ...],
          "MSFT": [{"Date": "2023-01-31", "Open": 250.089996, "High": 256.25, "Low": 242.529999, "Close": 252.509995, "Adj Close": 251.873795, "Volume": 47146900}, ...]}

    Args:
        tickers: A list of stock ticker symbols (e.g., ["AAPL", "MSFT"]).
        start_date: The start date for the data (e.g., "2023-01-01").
        end_date: The end date for the data in YYYY-MM-DD format (e.g., "2023-12-31").
        metric:  If "all", returns all available data columns (Open, High, Low, Close, Volume).
            Otherwise, specifies a single metric to return (e.g., "Close"). Defaults to "all".
        sampling: The frequency of the data. Can be "daily", "weekly", or "monthly". Defaults to "monthly".

    Returns:
        dict: A dictionary where keys are ticker symbols and values are lists of historical data records.
             Each record is a dictionary containing 'Date' and the requested metrics.

    Raises:
        ValueError: If an invalid sampling frequency is provided.


    """

    df = yf.download(tickers, start=start_date, end=end_date)

    if metric != "all":
        df = df[metric]

    if sampling == "weekly":
        df = df.resample("W-SAT").last()
    elif sampling == "monthly":
        df = df.resample("ME").last()
    elif sampling == "quarterly":
        df = df.resample("QE").last()
    elif sampling == "daily":
        pass
    else:
        raise ValueError(
            "Invalid sampling frequency. Use 'daily', 'weekly', 'monthly', 'quarterly."
        )

    result = {}
    for ticker in tickers:
        if metric == "all":
            df_tick = df.loc[:, (slice(None), ticker)]
            df_tick.columns = df_tick.columns.droplevel("Ticker")
        else:
            df_tick = df.loc[:, ticker]
            df_tick = df_tick.to_frame(name=metric)
        df_tick = df_tick.reset_index()
        df_tick["Date"] = df_tick["Date"].dt.strftime("%Y-%m-%d")
        result[ticker] = df_tick.to_dict(orient="records")

    return result


@tool
def get_fred_data(
    series: str, start_date: str, end_date: str, sampling: str = "monthly"
) -> list[dict]:
    """Downloads data from the Federal Reserve Economic Data (FRED) database and returns it as dictionary.

    Examples:
        >>> get_fred_data("GDP", "2023-01-01", "2023-01-10")
        [{"Date": "2023-01-01", "GDP": 21.0}, {"Date": "2023-01-02", "GDP": 22.0}, ...]

    Args:
        series: The FRED series ID (e.g., "GDP").
        start_date: The start date for the data (e.g., "2023-01-01").
        end_date: The end date for the data in YYYY-MM-DD format (e.g., "2023-12-31").
        sampling: The frequency of the data. Can be "monthly", "quarterly", or "yearly". Defaults to "monthly".

    Returns:
        list: A list representing a list of dictionaries, where each dictionary contains 'Date' and the value of the FRED series.

    Raises:
        ValueError: If an invalid sampling frequency is provided.


    """
    df = pdr.data.DataReader(series, start=start_date, end=end_date, data_source="fred")

    if sampling == "monthly":
        df = df.resample("ME").last()
    elif sampling == "quarterly":
        df = df.resample("QE").last()
    elif sampling == "yearly":
        df = df.resample("YE").last()
    else:
        raise ValueError(
            "Invalid sampling frequency. Use 'monthly', 'quarterly', or 'yearly'."
        )

    df.reset_index(inplace=True)
    df["DATE"] = df["DATE"].dt.strftime("%Y-%m-%d")
    df.rename(columns={"DATE": "Date"}, inplace=True)

    result = df.to_dict(orient="records")

    return result


@tool
def run_ols_regression(y: List[float], X: List[float]) -> dict:
    """Runs a simple Ordinary Least Squares (OLS) regression.
    I using to compute beta, make sure the dates are aligned.

    Examples:
        >>> y = [1, 2, 3, 4, 5]
        >>> X = [2, 4, 5, 4, 5]
        >>> run_ols_regression(y, X)
        {"const": -0.4, "coef": 0.9}

    Args:
        y: The dependent variable.
        X: The independent variable(s).

    Returns:
        dict: A dictionary containing the constant and coefficient of the OLS regression.


    """
    X = np.array(X)
    y = np.array(y)
    X = sm.add_constant(X)
    model = sm.OLS(y, X)
    results = model.fit()
    params = results.params
    const, coef = params
    return {"const": const, "coef": coef}

In [ ]:
stock_analysis_agent = CodeAgent(
    tools=[get_ticker_data, get_fred_data, run_ols_regression, DuckDuckGoSearchTool()],
    model=model,
    name="stock_analyst_agent",
    description="A research agent that specializes in analyzing stock performance, computing technical indicators, and forecasting volatility.",
)

In [ ]:
stock_analysis_agent.run("What immediate impact did Amazon's announcment of Alexa+ have on its stock price?")

In [ ]:
stock_analysis_agent.run("How has Amazon's stock price changed since the Fed began lowering rates")

In [ ]:
stock_analysis_agent.run("Compare and analyze the market beta for FAANG stocks since 2019")